In [1]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
import numpy as np
from faker import Faker
from typing import List

In [2]:
client = QdrantClient(host="localhost", port=6333)

In [3]:
my_collection = "first_collection"
client.create_collection(
    collection_name=my_collection,
    vectors_config=models.VectorParams(size=100, distance=models.Distance.COSINE)
)

True

In [4]:
data = np.random.uniform(low=0.0, high=1.0, size=(1_000, 100))
index: List = list(range(1_000))
index[-10:]

[990, 991, 992, 993, 994, 995, 996, 997, 998, 999]

In [5]:
client.upsert(
    collection_name=my_collection,
    points=models.Batch(
        ids=index,
        vectors=data.tolist(),
    )
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [6]:
client.retrieve(
    collection_name=my_collection,
    ids=[10,14,500],
)

[Record(id=14, payload={}, vector=None, shard_key=None),
 Record(id=10, payload={}, vector=None, shard_key=None),
 Record(id=500, payload={}, vector=None, shard_key=None)]

In [7]:
fakerInstance = Faker()

payload: List[models.Payload] = []

for i in range(1_000):
    payload.append({
        "artist": fakerInstance.name(),
        "song": " ".join(fakerInstance.words()),
        "url_song": fakerInstance.url(),
        "year": fakerInstance.year(),
        "country": fakerInstance.country()
    })
    
payload[:2]

[{'artist': 'Brandon Smith',
  'song': 'term foot into',
  'url_song': 'https://williams.biz/',
  'year': '1987',
  'country': 'Bolivia'},
 {'artist': 'Richard Barnes',
  'song': 'stock positive smile',
  'url_song': 'http://dennis-velazquez.org/',
  'year': '2012',
  'country': 'Kiribati'}]

In [8]:
client.upsert(
    collection_name=my_collection,
    points=models.Batch(
        ids=index,
        vectors=data.tolist(),
        payloads=payload
    )
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

## Semantic Search

In [9]:
living_la_vida_loca = np.random.uniform(low=-1.0, high=1.0, size=(100)).tolist()
living_la_vida_loca[:5]

[-0.26216005629485917,
 -0.901491149448618,
 0.045003041951861666,
 -0.9960473680032687,
 0.29745093447643267]

In [10]:
client.search(
    collection_name=my_collection,
    query_vector=living_la_vida_loca,
    limit=5
)

[ScoredPoint(id=698, version=1, score=0.32491204, payload={'artist': 'Jeffrey Ramsey', 'country': 'Guinea-Bissau', 'song': 'other paper program', 'url_song': 'http://moss.com/', 'year': '2013'}, vector=None, shard_key=None),
 ScoredPoint(id=504, version=1, score=0.27215195, payload={'artist': 'Dawn Juarez', 'country': 'Switzerland', 'song': 'southern PM expect', 'url_song': 'https://www.carter.com/', 'year': '1980'}, vector=None, shard_key=None),
 ScoredPoint(id=314, version=1, score=0.26946408, payload={'artist': 'Joshua Booth', 'country': 'Oman', 'song': 'herself organization hospital', 'url_song': 'https://holland.com/', 'year': '2011'}, vector=None, shard_key=None),
 ScoredPoint(id=179, version=1, score=0.26808584, payload={'artist': 'Bobby Hale', 'country': 'Kiribati', 'song': 'put also next', 'url_song': 'http://www.berry-cortez.org/', 'year': '1989'}, vector=None, shard_key=None),
 ScoredPoint(id=168, version=1, score=0.26779476, payload={'artist': 'Evan Hawkins MD', 'country': 